In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

df = pd.read_csv('championsdata.csv')
df.head(5)

,Year,Team,Game,Win,Home,MP,FG,FGA,FGP,TP,...,FTP,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1980,Lakers,1,1,1,240,48,89,0.539,0,...,0.867,12,31,43,30,5,9,17,24,109
1,1980,Lakers,2,0,1,240,48,95,0.505,0,...,0.667,15,37,52,32,12,7,26,27,104
2,1980,Lakers,3,1,0,240,44,92,0.478,0,...,0.767,22,34,56,20,5,5,20,25,111
3,1980,Lakers,4,0,0,240,44,93,0.473,0,...,0.737,18,31,49,23,12,6,19,22,102
4,1980,Lakers,5,1,1,240,41,91,0.451,0,...,0.788,19,37,56,28,7,6,21,27,108


In [2]:
df["Win"].corr(df["FGA"])

-0.07050296028062243

In [3]:
df["Win"].corr(df["Home"])

0.22600839282129584

In [4]:
df["Win"].corr(df["FGP"])

0.4187762683046484

In [5]:
df["Win"].corr(df["PTS"])

0.3950080977481507

In [6]:
df.corr()
#Our main features for determining a win
#Will be FGP, FT(MADE), DRB, AND POINTS

,Year,Game,Win,Home,MP,FG,FGA,FGP,TP,TPA,...,FTP,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Year,1.000000,-0.013034,0.018865,0.017833,0.038076,-0.384162,-0.351961,-0.189437,0.765864,0.826494,...,0.013352,-0.375680,0.141763,-0.162384,-0.366905,-0.120682,-0.060679,-0.295238,-0.246790,-0.210735
Game,-0.013034,1.000000,0.096004,-0.154457,-0.112216,-0.105332,-0.082260,-0.062538,0.039666,0.025935,...,-0.004889,0.018762,-0.053816,-0.027145,-0.126379,-0.006437,-0.133452,-0.020670,0.124912,-0.039981
Win,0.018865,0.096004,1.000000,0.226008,-0.115526,0.242698,-0.070503,0.418776,0.036517,-0.085514,...,0.207009,-0.091179,0.401033,0.237150,0.208196,0.131785,0.092909,-0.224392,-0.146904,0.395008
Home,0.017833,-0.154457,0.226008,1.000000,0.064854,0.148059,0.080108,0.122578,0.028553,0.026015,...,0.024942,0.045662,0.302585,0.260986,0.234745,0.022109,0.102567,-0.058148,-0.148750,0.192108
MP,0.038076,-0.112216,-0.115526,0.064854,1.000000,0.182213,0.377379,-0.094893,0.072193,0.067871,...,0.006539,0.159941,0.004993,0.118427,0.055988,0.073683,0.020606,0.028306,0.160658,0.156821
FG,-0.384162,-0.105332,0.242698,0.148059,0.182213,1.000000,0.717653,0.716452,-0.161134,-0.301150,...,0.067563,0.214147,0.097367,0.226968,0.726910,0.260889,0.115427,0.033195,0.276572,0.844687
FGA,-0.351961,-0.082260,-0.070503,0.080108,0.377379,0.717653,1.000000,0.034344,-0.183846,-0.191084,...,0.057074,0.600364,0.068712,0.482227,0.469553,0.264415,0.094237,0.013656,0.335417,0.552078
FGP,-0.189437,-0.062538,0.418776,0.122578,-0.094893,0.716452,0.034344,1.000000,-0.034193,-0.232673,...,0.044093,-0.289214,0.063192,-0.159667,0.567209,0.115858,0.062885,0.032326,0.070499,0.664525
TP,0.765864,0.039666,0.036517,0.028553,0.072193,-0.161134,-0.183846,-0.034193,1.000000,0.888239,...,-0.001108,-0.363377,0.068601,-0.208753,-0.058553,-0.080939,-0.088313,-0.278003,-0.157905,0.066088
TPA,0.826494,0.025935,-0.085514,0.026015,0.067871,-0.301150,-0.191084,-0.232673,0.888239,1.000000,...,0.003024,-0.300144,0.066792,-0.164787,-0.212537,-0.108574,-0.093788,-0.286712,-0.154606,-0.111820


In [11]:
X= df[['PTS', 'DRB', 'FT', 'FGP']]  
Y= df[['Win']]  

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.4,random_state=100)

logreg= LogisticRegression()
logreg.fit(X_train,y_train)

y_pred=logreg.predict(X_test)
print (X_test) #test dataset
print (y_pred) #predicted values

     PTS  DRB  FT    FGP
59   105   29  15  0.456
124  108   33  32  0.451
127  106   33  24  0.544
198  108   37  20  0.443
81    83   21  13  0.390
..   ...  ...  ..    ...
73    97   22  15  0.521
195  111   24  26  0.594
57   105   27  35  0.486
44   107   31  23  0.482
165  104   16  16  0.513

[88 rows x 4 columns]
[1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 1 1 0 1 1 1 0]


C:\Users\darin\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
from sklearn import metrics
from sklearn.metrics import classification_report
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("Recall: ",metrics.recall_score(y_test, y_pred, zero_division=1))
print("Precision: ",metrics.precision_score(y_test, y_pred, zero_division=1))
print("CL Report: ",metrics.classification_report(y_test, y_pred, zero_division=1))

Accuracy:  0.7954545454545454
Recall:  0.9016393442622951
Precision:  0.8208955223880597
CL Report:                precision    recall  f1-score   support

           0       0.71      0.56      0.63        27
           1       0.82      0.90      0.86        61

    accuracy                           0.80        88
   macro avg       0.77      0.73      0.74        88
weighted avg       0.79      0.80      0.79        88

